In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv, DataFrame, concat
from statsmodels.tsa.stattools import acf, q_stat, adfuller, kpss
import csv
from torch.utils.data import TensorDataset, DataLoader
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error

In [ ]:
train_res_lst = []
test_res_lst = []

In [ ]:
class EarlyStopper():
    def __init__(self, patience=5, min_delta=0, filename='optimal_weight.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_val_loss = np.inf
        self.filename = filename

    def early_stop(self, model, val_loss):
        if val_loss < self.min_val_loss:
            self.min_val_loss = val_loss
            torch.save(model.state_dict(), self.filename)
            self.counter = 0
        elif val_loss > (self.min_val_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                model.load_state_dict(torch.load(self.filename))
                return True
        return False

In [ ]:
class MLP_Model(nn.Module):

    def __init__(self, dim_features, seq_length):
        super(MLP_Model, self).__init__()
        self.input_layer = nn.Linear(dim_features*seq_length, 100, bias=False)
        self.tanh = nn.Tanh()
        self.hidden_layer_1 = nn.Linear(100, 8, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.hidden_layer_2 = nn.Linear(8, 50, bias=False)
        self.output_layer = nn.Linear(50, 1, bias=False)

        self.flatten = nn.Flatten()

        self.train_loss_lst = []
        self.test_loss_lst = []


    def forward(self, x):
        x = self.flatten(x)
        x = self.input_layer(x)
        x = self.tanh(x)
        x = self.hidden_layer_1(x)
        x = self.sigmoid(x)

        # in the following part, it will be replaced by the dlem model
        x = self.hidden_layer_2(x)
        x = self.tanh(x)
        x = self.output_layer(x)

        return x

    def dense_forward(self, x):
        x = self.flatten(x)
        x = self.input_layer(x)
        x = self.tanh(x)
        x = self.hidden_layer_1(x)
        x = self.sigmoid(x)

        return x



    def train(self, train_loader, test_loader, early_stopper, epochs, learning_rate=1e-4):

        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        criterion = torch.nn.MSELoss()

        for ix in range(epochs):
            tmp_train_loss = []
            tmp_test_loss = []
            for inputs, val in train_loader:
                optimizer.zero_grad()
                output = self.forward(inputs)
                loss = criterion(val, output)
                loss.backward()
                optimizer.step()
                tmp_train_loss.append(loss.item())

            for test_inputs, val in test_loader:
                output = self.forward(test_inputs)
                test_loss = criterion(val, output)
                tmp_test_loss.append(test_loss.item())


            print("Epoch ", ix+1, "Train Loss : ", np.mean(tmp_train_loss), \
                  " , Test Loss : ", np.mean(tmp_test_loss))

            self.train_loss_lst.append(np.mean(tmp_train_loss))
            self.test_loss_lst.append(np.mean(tmp_test_loss))

            if early_stopper.early_stop(self, np.mean(tmp_test_loss)):
                print("early stopping is true")
                return self.train_loss_lst, self.test_loss_lst

        return self.train_loss_lst, self.test_loss_lst

In [ ]:
class Scaler():
    def __init__(self):
        super(Scaler, self).__init__()
        # from the left to the right, it is doy, fh2o, ta, h2o, sm, st, n2o
        self.min_val = torch.tensor([[170,  0, 0,  0, 30, 10, 0]])
        self.max_val = torch.tensor([[295,  5, 40, 50, 50, 25, 260]])

    def reverse_transform(self, input_data):
        output_data = (self.max_val - self.min_val) * input_data + self.min_val
        return output_data

    def transform(self, input_data):
        output_data = (input_data - self.min_val)/(self.max_val - self.min_val)
        return output_data

    def exponential_transform(self, input_data, alpha):
        return torch.exp(alpha*input_data)

    def reverse_n2o(self, n2o_predicted):
        n2o_output = (self.max_val[:, -1] - self.min_val[:, -1]) * n2o_predicted + self.min_val[:, -1]
        return n2o_output

In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series, autolag='t-stat', maxlag = 100, regression='n') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    #print("Result : ", result)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if result[1] <= 0.005:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")


def kpss_test(timeseries, title=''):
    print(f'Results of KPSS Test for {title} : ')
    kpsstest = kpss(timeseries, regression="ct", nlags=500)
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output.to_string())

    if kpsstest[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data is not trend stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data is trend stationary")

In [ ]:
def calculate_metric(original, predicted, title):
    print(title)
    rmse = mean_squared_error(original, predicted, squared=False)
    print('Test RMSE: %.6f' % rmse)
    mae = mean_absolute_error(original, predicted)
    print('Test MAE: %.6f' % mae)
    mape = mean_absolute_percentage_error(original, predicted)
    print('Test MAPE: %.6f' % mape)
    r2 = r2_score(original, predicted)
    print('Test R2: %.6f' % r2)
    residue_err = original - predicted
    print("--------------")
    adf_test(residue_err, title)
    kpss_test(residue_err, title)
    print("--------------")
    print('Test Residue Mean: %.6f' %np.mean(residue_err))
    print('Test Residue std: %.6f' %np.std(residue_err))
    print('Test Residue var: %.6f' %np.var(residue_err))

    acc_rate = np.sum(predicted)/np.sum(original)
    print("Sum of the Emission FN2O (Measured) : %6f" %np.sum(original))
    print("Sum of the Emission FN2O (Predicted): %.6f" %np.sum(predicted))
    print("The percentage of the accuracy rate (p/m): %6f" %acc_rate)

    return np.array([rmse, mae, mape, r2, acc_rate])

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [ ]:
# prepare the training data
#broadcast_side_2021 = pd.read_csv("2021_broadcast_side_all.csv")
broadcast_side_2021 = pd.read_csv("https://raw.githubusercontent.com/noobstang/NNtraining/master/Weather49Sets/weatherstats_ottawa_daily.csv")
broadcast_side_2021.drop('Unnamed: 0', axis=1, inplace=True)
#broadcast_side_2021.drop('DOY', axis=1, inplace=True)
broadcast_side_2021 = broadcast_side_2021[['DOY','FH2O', 'TA', 'H2O', 'SM', 'ST', 'FN2O']]
print(broadcast_side_2021[500:4500])

<ipython-input-10-0c8287f36d43>:3: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  broadcast_side_2021 = pd.read_csv("https://raw.githubusercontent.com/noobstang/NNtraining/master/Weather49Sets/weatherstats_ottawa_daily.csv")


KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
# prepare the training data
broadcast_side_2022 = pd.read_csv("2022_broadcast_side_all.csv")
broadcast_side_2022.drop('Unnamed: 0', axis=1, inplace=True)
#broadcast_side_2021.drop('DOY', axis=1, inplace=True)
broadcast_side_2022 = broadcast_side_2022[['DOY', 'FH2O','TA', 'H2O', 'SM', 'ST', 'FN2O']]
broadcast_side_2022['DOY'] = broadcast_side_2022['DOY']
print(broadcast_side_2022[0:4000])

In [ ]:
scaler = Scaler()

test_data = scaler.transform(torch.tensor(broadcast_side_2022[0:4000].values))
train_data = scaler.transform(torch.tensor(broadcast_side_2021[500:4500].values))

In [ ]:
test_data

In [ ]:
test_n2o = scaler.exponential_transform(train_data[:, 0], alpha=-3) - 0.07
train_n2o = scaler.exponential_transform(test_data[:, 0], alpha=-3)

# plot history
plt.figure(figsize=(18, 12))
#plt.plot(res.detach().numpy(), label='train actual')
#plt.plot(train_y, label='train predict')

plt.plot(test_n2o, label='test actual')
plt.plot(train_n2o, label='train actual')
plt.xlabel("Time")
plt.ylabel("Nitrate")
plt.legend()
plt.show()

In [ ]:
test_data[:,0] = test_n2o

train_data[:,0] = train_n2o


In [ ]:
# specify the number of lag steps
# prepare the training data for inputting the LSTM model
n_steps = 5
n_features = 6
n_out = 1
train_x = series_to_supervised(train_data[:, 0:6], n_steps)
test_x = series_to_supervised(test_data[:, 0:6], n_steps)

In [ ]:
train_x.shape

In [ ]:
train_x = train_x.reshape(train_x.shape[0], n_steps, n_features)
test_x = test_x.reshape(test_x.shape[0], n_steps, n_features)
train_x.shape

In [ ]:
y_2021_f = train_data[:, -1]
y_2022_f = test_data[:, -1]
y_2021 = y_2021_f[n_steps:]
y_2022 = y_2022_f[n_steps:]
y_2021 = y_2021.reshape(4000-n_steps, 1)
y_2022 = y_2022.reshape(4000-n_steps, 1)

In [ ]:
test_y = broadcast_side_2022[n_steps:4000]['FN2O']
train_y = broadcast_side_2021[500+n_steps:4500]['FN2O']

test_y = test_y.values.reshape(4000-n_steps)
train_y = train_y.values.reshape(4000-n_steps)

In [ ]:
batch_size = 20
dataset_2021 = TensorDataset(torch.tensor(train_x, dtype=torch.float32), \
                           torch.tensor(y_2021, dtype=torch.float32))
loader_2021 = DataLoader(dataset_2021, shuffle=True, batch_size=batch_size)

dataset_2022 = TensorDataset(torch.tensor(test_x, dtype=torch.float32), \
                           torch.tensor(y_2022, dtype=torch.float32))
loader_2022 = DataLoader(dataset_2022, shuffle=True, batch_size=batch_size)

In [ ]:
mlp = MLP_Model(n_features, n_steps)

In [ ]:
# Print model's state_dict
#print("Generator's state_dict:")
for param_tensor in mlp.state_dict():
    print(param_tensor, "\t", mlp.state_dict()[param_tensor].size())

sum(p.numel() for p in mlp.parameters() if p.requires_grad)

In [ ]:
early_stopper = EarlyStopper(20, 0, filename='mlp_optimal_weight.pth')
train_lss, test_lss = mlp.train(loader_2021, loader_2022, early_stopper, epochs=100, learning_rate=1e-4)

In [ ]:
res_2022_n = mlp.forward(torch.tensor(test_x).float())
res_2021_n = mlp.forward(torch.tensor(train_x).float())
res_2022 = scaler.reverse_n2o(res_2022_n).reshape(4000-n_steps)
res_2021 = scaler.reverse_n2o(res_2021_n).reshape(4000-n_steps)

In [ ]:
width = 40
height = 33
plt.rcParams.update({'font.size': 50})
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(width, height))

ax[0].plot(res_2022.detach().numpy(), color='orangered', label='2022 Predicted Emission')
ax[0].plot(test_y, color='cyan', label='2022 Measured Emission')
ax[0].legend()
ax[0].set_title("Flux of N2O for 2022 Growing Season (MLP, 5 Steps)")
ax[0].set_xlabel("Time Steps")
ax[0].set_ylabel("$nmol \ m^{-2} s^{-1}$")


ax[1].plot(res_2021.detach().numpy(), color='orangered', label='2021 Predicted Emission')
ax[1].plot(train_y, color='cyan', label='2021 Measured Emission')
ax[1].legend()
ax[1].set_title("Flux of N2O for 2021 Growing Season (MLP, 5 Steps)")
ax[1].set_xlabel("Time Steps")
ax[1].set_ylabel("$nmol \ m^{-2} s^{-1}$")


f.tight_layout()
f.show()

In [ ]:
train_result = calculate_metric(train_y, res_2021.detach().numpy(), "2021 growing season (MLP, 5 Steps)")
train_res_lst.append(train_result)
print("****************************************************************************")
test_result = calculate_metric(test_y, res_2022.detach().numpy(), "2022 growing season (MLP, 5 Steps)")
test_res_lst.append(test_result)
print("-----------------------------------------------------------------------------")

In [ ]:
#wfp, P_clay, Rh, kden, temp,  k_nit, D_NO3, D_NH4
dense_test  = mlp.dense_forward(torch.tensor(test_x).float()).detach().numpy()

dense_train  = mlp.dense_forward(torch.tensor(train_x).float()).detach().numpy()


width = 100
height = 33
plt.rcParams.update({'font.size': 50})
f, ax = plt.subplots(nrows=2, ncols=4, figsize=(width, height))

ax[0][0].plot(dense_train[:,0], color='orangered', label='Training Dataset')
ax[0][0].plot(dense_test[:,0], color='cyan', label='Testing Dataset')
ax[0][0].legend()
ax[0][0].set_title("Output of Node 1")
ax[0][0].set_xlabel("Time Steps")


ax[0][1].plot(dense_train[:,1], color='orangered', label='Training Dataset')
ax[0][1].plot(dense_test[:,1], color='cyan', label='Testing Dataset')
ax[0][1].legend()
ax[0][1].set_title("Output of Node 2")
ax[0][1].set_xlabel("Time Steps")

ax[0][2].plot(dense_train[:,2], color='orangered', label='Training Dataset')
ax[0][2].plot(dense_test[:,2], color='cyan', label='Testing Dataset')
ax[0][2].legend()
ax[0][2].set_title("Output of Node 3")
ax[0][2].set_xlabel("Time Steps")


ax[0][3].plot(dense_train[:,3], color='orangered', label='Training Dataset')
ax[0][3].plot(dense_test[:,3], color='cyan', label='Testing Dataset')
ax[0][3].legend()
ax[0][3].set_title("Output of Node 4")
ax[0][3].set_xlabel("Time Steps")


ax[1][0].plot(dense_train[:,4], color='orangered', label='Training Dataset')
ax[1][0].plot(dense_test[:,4], color='cyan', label='Testing Dataset')
ax[1][0].legend()
ax[1][0].set_title("Output of Node 5")
ax[1][0].set_xlabel("Time Steps")

ax[1][1].plot(dense_train[:,5], color='orangered', label='Training Dataset')
ax[1][1].plot(dense_test[:,5], color='cyan', label='Testing Dataset')
ax[1][1].legend()
ax[1][1].set_title("Output of Node 6")
ax[1][1].set_xlabel("Time Steps")

ax[1][2].plot(dense_train[:,6], color='orangered', label='Training Dataset')
ax[1][2].plot(dense_test[:,6], color='cyan', label='Testing Dataset')
ax[1][2].legend()
ax[1][2].set_title("Output of Node 7")
ax[1][2].set_xlabel("Time Steps")

ax[1][3].plot(dense_train[:,7], color='orangered', label='Training Dataset')
ax[1][3].plot(dense_test[:,7], color='cyan', label='Testing Dataset')
ax[1][3].legend()
ax[1][3].set_title("Output of Node 8")
ax[1][3].set_xlabel("Time Steps")

f.tight_layout()
f.show()

In [ ]:
print("2021 Train Res (MLP, 5 Steps) : ", train_res_lst)
print("Train Mean : ", np.mean(train_res_lst, axis=0))
print("Train Standard : ", np.std(train_res_lst, axis=0))

In [ ]:
print("2022 Test Res (MLP, 5 Steps): ", test_res_lst)
print("Test Mean : ", np.mean(test_res_lst, axis=0))
print("Test Standard : ", np.std(test_res_lst, axis=0))